In [1]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
import matplotlib.image as mpimg
import matplotlib
from CNN_data_preprocess import get_label_mapping, get_files, bottle_neck_process
slim = tf.contrib.slim
from color_filter import red_filtered, blue_filtered
import cv2

In [2]:
checkpoint_file = './model_file/inception_resnet_v2_2016_08_30.ckpt'
label_file = './labels.txt'
train_file = './Data9_10_sum/'
bottle_neck_file = './middle_data/bottle_neck9_10_sum/'
save_model_file = './model_file/inception_resnet_v2_for9_10_sum'
last_layer_file = './middle_data/last_layer9_10_sum/'

In [3]:
# def build_graph():
images = tf.placeholder(tf.float32, shape=(None, 480, 720, 3))
labels = tf.placeholder(tf.int32, shape=(None,))

# restore from the inception_resnet model
with slim.arg_scope(inception_resnet_v2_arg_scope()):
        _, end_points = inception_resnet_v2(images, num_classes = 1001, is_training = False) 

# extract the before_logit tensor from the extracted model
before_logit = end_points['PreLogitsFlatten']


# define a placeholder for training from the bottle, not from the beginning.
bottle_neck = tf.placeholder(tf.float32, shape=(None,1536 * 2))

# now define the fine tune part of the graph
with tf.name_scope('my_fine_tune') as scope:
#     dropout = tf.layers.dropout(bottle_neck, rate=0.3, name='dropout')
    batch_norm1 = tf.layers.batch_normalization(bottle_neck, name='batch_norm1')
    dropout1 = tf.layers.dropout(batch_norm1, rate=0.3, name='dropout1')
    dense1 = tf.layers.dense(dropout1, 512, activation=tf.nn.relu, name='dense1')
    batch_norm2 = tf.layers.batch_normalization(dense1, name='batch_norm2')
    dropout2 = tf.layers.dropout(batch_norm2, rate = 0.3, name='dropout2')
    dense2 = tf.layers.dense(dropout2, 128, activation=tf.nn.relu, name='dense2')
    batch_norm3 = tf.layers.batch_normalization(dense2, name='batch_norm3')
    dropout3 = tf.layers.dropout(batch_norm3, rate = 0.3, name='dropout3')
    logits = tf.layers.dense(dropout3, 3, name='logits')

    x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
    loss = tf.reduce_mean(x_entropy, name='loss')
    correct = tf.nn.in_top_k(logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    # extract variables that need to be trained
    weight1, bias1 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'dense1')
    weight2, bias2 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'logits')
    variables_to_train = [weight1, weight2, bias1, bias2]

    optimizer = tf.train.AdamOptimizer()
    train_op = slim.learning.create_train_op(loss, optimizer, variables_to_train=variables_to_train)

# defiine the name scope we don't need to restore from inception_resnet
exclude = ['InceptionResnetV2/AuxLogits', 'InceptionResnetV2/Logits',
           'my_fine_tune/','dense','logits','batch_norm']

variables_to_restore = slim.get_variables_to_restore(exclude = exclude)
saver = tf.train.Saver(variables_to_restore)
init = tf.global_variables_initializer()
builder = tf.saved_model.builder.SavedModelBuilder(save_model_file)

#     return images, labels, before_logit, bottle_neck,batch_norm2, loss, accuracy, train_op, saver, init, builder


In [4]:
print(before_logit)

Tensor("InceptionResnetV2/Logits/Dropout/Identity:0", shape=(?, 1536), dtype=float32)


In [5]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

In [6]:
# extract the data before the fine tune layer
def get_bottle_neck_data(sess, before_logit, images, train_file, bottle_neck_file, label_file, hsv = False):
    id2label, label2id = get_label_mapping(label_file)
    for i in id2label:
        read_dir = train_file + i
        des_dir = bottle_neck_file + i + '/'
        os.mkdir(des_dir)
        
        files = get_files(read_dir)
        for f in files:
            img = mpimg.imread(f).reshape(1,480,720,3) / 255
            img1 = cv2.imread(f)
            img2=red_filtered(img1)[1].reshape(1,480,720,3) / 255
            img3=blue_filtered(img1)[1].reshape(1,480,720,3) / 255
            if(hsv):
                img = matplotlib.colors.rgb_to_hsv(img)
            bottle = sess.run(before_logit, feed_dict={images:img})
            bottle1 = sess.run(before_logit, feed_dict={images:img2})
            bottle2 = sess.run(before_logit, feed_dict={images:img3})
            bottle = np.concatenate((bottle, bottle1), axis = 1)
            bottle = np.concatenate((bottle, bottle2), axis = 1)
            print(bottle.shape)
            np.save(des_dir + f[len(read_dir):],  bottle)
            print('save the bottle neck file ' + f)
            

In [7]:
def get_bottle_neck_data_timestamp(sess, before_logit, images, train_file, bottle_neck_file, label_file, hsv = False):
    os.mkdir(bottle_neck_file)
    folders = listdir_nohidden(train_file)
    print(bottle_neck_file)
    for folder in folders:
        train_actual = train_file + folder + '/'
        bottle_neck_actual = bottle_neck_file + folder + '/'
        print(train_actual)
        print(bottle_neck_actual)
        os.mkdir(bottle_neck_actual)
        get_bottle_neck_data(sess, before_logit, images, train_actual, bottle_neck_actual, label_file, hsv = False)

In [8]:
def get_accuracy(sess):
    sess.run(init)
    saver.restore(sess, checkpoint_file)
    
    bottle_neck_processor = bottle_neck_process(bottle_neck_file, label_file) 

    val_loss, val_accuracy = sess.run([loss, accuracy],
                                              feed_dict={bottle_neck: bottle_neck_processor.val_inputs,
                                                         labels: bottle_neck_processor.val_labels})
    
    for i in bottle_neck_processor.id2label:
        val_accuracy = sess.run([accuracy],
                              feed_dict={bottle_neck: bottle_neck_processor.val_inputs_each[i],
                                         labels: bottle_neck_processor.val_labels_each[i]})
        print(i + ': the val accuracy is: '  + str(val_accuracy))

In [ ]:
def train_graph(sess):
#     images, labels, before_logit, bottle_neck, batch_norm2, loss, accuracy, train_op, saver, init, builder = model_list
    
    sess.run(init)
    saver.restore(sess, checkpoint_file)

    # if there is no dir for the bottle data, then get the data first.
    if(os.path.isdir(bottle_neck_file) is not True):
        os.mkdir(bottle_neck_file)
        get_bottle_neck_data(sess, before_logit, images, train_file, bottle_neck_file, label_file, hsv = False)

    # get training data from bottle_neck file, and train.
    bottle_neck_processor = bottle_neck_process(bottle_neck_file, label_file) 
    for i in range(2000):
        inputs, groud_truth = bottle_neck_processor.next_batch(256)
        train_loss, train_accuracy = sess.run([train_op, accuracy], feed_dict={bottle_neck: inputs, 
                                                                             labels: groud_truth})

        if(i % 50 == 0):
            val_loss, val_accuracy = sess.run([loss, accuracy],
                                              feed_dict={bottle_neck: bottle_neck_processor.val_inputs,
                                                         labels: bottle_neck_processor.val_labels})
        
            print('####################################')
            print('the training loss after ' + str(i) + ' iterations is: ' + str(train_loss))
            print('the training accuracy after ' + str(i) + ' iterations is: ' + str(train_accuracy))
            print('the val loss after ' + str(i) + ' iterations is: ' + str(val_loss))
            print('the val accuracy after ' + str(i) + ' iterations is: ' + str(val_accuracy))
            print('-------------')
            
            for i in bottle_neck_processor.id2label:
                val_accuracy = sess.run([accuracy],
                                      feed_dict={bottle_neck: bottle_neck_processor.val_inputs_each[i],
                                                 labels: bottle_neck_processor.val_labels_each[i]})
                print(i + ': the val accuracy is: '  + str(val_accuracy))
        
#         after the training process finished, store the model to a '.pb' version file.
    builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.TRAINING],
                                         signature_def_map=None, assets_collection=None)
    
    builder.save()

In [ ]:
if __name__ == '__main__':
    config = tf.ConfigProto()
    config.allow_soft_placement = True
    with tf.Session(config=config) as sess:
        train_graph(sess)
#         get_accuracy(sess)
#         sess.run(init)
#         saver.restore(sess, checkpoint_file)
#         get_bottle_neck_data_timestamp(sess, before_logit, images, train_file, bottle_neck_file, label_file, hsv = False)

INFO:tensorflow:Restoring parameters from ./model_file/inception_resnet_v2_2016_08_30.ckpt
./middle_data/bottle_neck9_10_sum/a
654
the lengeh of inputs is 654
./middle_data/bottle_neck9_10_sum/d
131
the lengeh of inputs is 131
./middle_data/bottle_neck9_10_sum/w
1924
the lengeh of inputs is 1924
####################################
the training loss after 0 iterations is: 1.14709
the training accuracy after 0 iterations is: 0.244094
the val loss after 0 iterations is: 0.885996
the val accuracy after 0 iterations is: 0.711111
-------------
a: the val accuracy is: [0.0]
d: the val accuracy is: [0.0]
w: the val accuracy is: [1.0]
####################################
the training loss after 50 iterations is: 0.432325
the training accuracy after 50 iterations is: 0.834646
the val loss after 50 iterations is: 0.495412
the val accuracy after 50 iterations is: 0.82963
-------------
a: the val accuracy is: [0.61538464]
d: the val accuracy is: [0.0]
w: the val accuracy is: [0.95833331]
#########

####################################
the training loss after 1100 iterations is: 0.144282
the training accuracy after 1100 iterations is: 0.944882
the val loss after 1100 iterations is: 0.459222
the val accuracy after 1100 iterations is: 0.855556
-------------
a: the val accuracy is: [0.73846155]
d: the val accuracy is: [0.42307693]
w: the val accuracy is: [0.92447919]
####################################
the training loss after 1150 iterations is: 0.158547
the training accuracy after 1150 iterations is: 0.944882
the val loss after 1150 iterations is: 0.476122
the val accuracy after 1150 iterations is: 0.844444
-------------
a: the val accuracy is: [0.69999999]
d: the val accuracy is: [0.1923077]
w: the val accuracy is: [0.9375]
####################################
the training loss after 1200 iterations is: 0.101127
the training accuracy after 1200 iterations is: 0.972441
the val loss after 1200 iterations is: 0.463425
the val accuracy after 1200 iterations is: 0.846296
-------------
